In [1]:
!pip install rtree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import dask.distributed
from shapely.geometry import Point, Polygon
import multiprocessing
from tqdm import tqdm
from rtree import index

In [14]:
pd.set_option('display.max_rows', 200)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp').filter(['Meshcode', 'geometry'])

# 標高データの空間結合

In [5]:
df_elevation = dd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/DEM/03215.csv', usecols=['X', 'Y', 'hyoukouchi'])

In [ ]:
#df_elevation = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/DEM/03215.csv', usecols=['X', 'Y', 'hyoukouchi'])

In [6]:
# # カラムを絞る
# df_elevation = df_elevation[['X', 'Y', 'hyoukouchi']]

# カラムの名称を変更
df_elevation = df_elevation.rename(columns={'X': 'longitude', 'Y': 'latitude'})


# Pointオブジェクトの作成
geometry = df_elevation.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1, meta=('geometry', 'object')).compute()


# GeoPandasデータフレームを作成
gdf_elevation = gpd.GeoDataFrame(df_elevation.compute(), geometry=geometry)

#gdf_elevationに座標系はついていないので付与
if gdf_elevation.crs is None:
	gdf_elevation.set_crs(epsg=4326, inplace=True)

# 不要なメモリを解放
del df_elevation, geometry

# 標高のデータを結合
gdf_osyu = gpd.sjoin(gdf_osyu_100m_mesh, gdf_elevation, how='left', op='contains')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [30]:
for i in gdf_osyu[gdf_osyu['Meshcode'] == '5840378485']['hyoukouchi']:
  print(i)

395.9
396.5
396.3
395.2
396.1
395.3
397.1
396.2
395.4
389.7
390.8
389.7
393.1
397.2
389.6
394.6


In [20]:
#標高の値をグループ演算で平均、最大、最低を求める
gdf_elevation = gdf_osyu.groupby('Meshcode').agg({'hyoukouchi':['mean', 'max', 'min']}).reset_index()

In [32]:
gdf_elevation[:1]

,Meshcode,hyoukouchi_mean,hyoukouchi_max,hyoukouchi_min,hyoukouchi_max-hyoukouchi_min
0,5840378485,394.29375,397.2,389.6,7.6


In [22]:
# カラム名を変更
gdf_elevation.columns = ['Meshcode', 'hyoukouchi_mean', 'hyoukouchi_max', 'hyoukouchi_min']

In [31]:
#標高の最大-最小のカラムを追加
gdf_elevation['hyoukouchi_max-hyoukouchi_min'] = gdf_elevation['hyoukouchi_max'] - gdf_elevation['hyoukouchi_min']

In [ ]:
gdf_elevation[gdf_elevation['hyoukouchi_max-hyoukouchi_min'] < 0]

,Meshcode,hyoukouchi_mean,hyoukouchi_max,hyoukouchi_min,hyoukouchi_max-hyoukouchi_min


In [ ]:
gdf_elevation.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市　標高/奥州市_標高.csv')